In [1]:
import torch
import json
import warnings

from torch.utils.data import Dataset, random_split
from transformers import AutoProcessor, AutoModelForImageTextToText, ProcessorMixin, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from pathlib import Path

Определим параметры модели, набора данных, обработчика и peft адаптеров

In [2]:
# Путь до набора данных
dataset_path = "../src_dataset_creator/dataset/AniDataset_t614"
# Модель для обучения с HuggingFace
model_id = "HuggingFaceTB/SmolVLM2-500M-Video-Instruct"
# Зададим устройство запуска как "auto" - huggingface обертка выберет доступное устройство автоматически
device = "auto"
# Выберем реализацию flash_attention
attn_implementation = "flash_attention_2" # sdpa, flex_attention, flash_attention_2
# Выберем желаемую точность вычислений
torch_dtype = torch.bfloat16
# Использовать ли LoRA для обучения модели
use_lora = True
use_qlora = False
# Максимальное количество кадров с видео (понизим с 64 до 32 для уменьшения занимаемого объема памяти)
max_frames = 32

In [3]:
# Подготовим конфиг LoRA
lora_config = None
bnb_config = None
if use_lora:
    lora_config = LoraConfig(
        r=8,
        lora_alpha=8,
        lora_dropout=0.1,
        target_modules=['down_proj','o_proj','k_proj','q_proj','gate_proj','up_proj','v_proj'],
        use_dora=True,
        inference_mode=False,
        init_lora_weights="gaussian",
    )
if use_qlora:
    bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )

In [4]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

## Загрузим небольшую модель VLM

In [6]:
# Загрузим модель
model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    attn_implementation=attn_implementation,
    quantization_config=bnb_config,
    device_map=device,
)
if lora_config is not None:
    # Применим LoRA к модели
    if use_qlora:
        model = prepare_model_for_kbit_training(model)
    # model = get_peft_model(model, lora_config)
    # Альтернативный способ применения Lora
    model.add_adapter(lora_config)
    model.enable_adapters()

# Выведем информацию о модели
print_trainable_parameters(model)
peak_mem = torch.cuda.max_memory_allocated()
print(f"The model as is is holding: {peak_mem / 1024**3:.2f} of GPU RAM")

trainable params: 507482304 || all params: 507482304 || trainable%: 100.00
The model as is is holding: 0.95 of GPU RAM


## Подготовим набор данных

Загрузим обработчик данных на входе/выходе модели

In [5]:
processor: ProcessorMixin = AutoProcessor.from_pretrained(model_id, use_fast=True)
# Изменим количество кадров
if max_frames:
    processor.video_processor.num_frames = max_frames

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


Определим класс набора данных

In [6]:
from typing import Any
from dataclasses import dataclass, asdict
from datetime import datetime


@dataclass
class AnimeData:
    id: str
    mal_id: str
    name: str
    title: str
    rating: str
    score: float
    released: datetime.date
    genres: list[str]
    main_characters: list[str]
    popularity: int
    description: str
    video_path: str

    def to_json(self) -> dict[str, Any]:
        data = asdict(self)
        data['released'] = data['released'].strftime("%Y-%m-%d %H:%M:%S")
        return data

    @classmethod
    def from_json(cls, data: dict[str, Any]) -> "AnimeData":
        data['released'] = datetime.strptime(data['released'], "%Y-%m-%d %H:%M:%S")
        return cls(**data)

In [7]:
class AnimeEpisodeCaptionDataset(Dataset):
    def __init__(
            self,
            dataset_path: str | Path
    ):
        dataset_path = Path(dataset_path)

        self.dataset_path = dataset_path

        # Загрузим аннотацию
        annotation_path = dataset_path / "annotation.json"
        with open(annotation_path, "r", encoding="utf-8") as f:
            anime_dataset = json.load(f)
        # Преобразуем информацию об элементе в класс данных
        anime_data: list[AnimeData] = [
            AnimeData.from_json(data)
            for data in anime_dataset['animes']
        ]
        # Проверим валидность данных
        for data in anime_data:
            self._validate_anime_data(data)

        self.anime_data = anime_data

    def _validate_anime_data(self, data: AnimeData):
        """ Проверка валидности данных об аниме """
        # Проверим, что видео доступно
        if not (video_path := Path(self.dataset_path, data.video_path)).exists():
            raise FileNotFoundError(
                f"No found video file at '{video_path}' for '{data.name}' title with id {data.mal_id}"
            )
        # Проверим, что описание не пустое
        if not data.description:
            raise ValueError(
                f"No found description for '{data.name}' title with id {data.mal_id}"
            )

    def get_anime_data_by_idx(self, item) -> AnimeData:
        return self.anime_data[item]

    def __getitem__(self, item) -> dict[str, list[dict[str, Any]]]:
        anime_data = self.anime_data[item]

        user_content = [
            {"type": "text", "text": "Caption the video. "},
            {"type": "video", "path": str(Path(self.dataset_path, anime_data.video_path))}
        ]
        if anime_data.main_characters:
            mc_info = ', '.join(anime_data.main_characters)
            user_content.insert(
                0,
                {"type": "text", "text": f"The main characters are {mc_info}. "}
            )
        if anime_data.genres:
            g_info = ', '.join(anime_data.genres)
            user_content.insert(
                0,
                {"type": "text", "text": f"The video genres are {g_info}. "}
            )
        assistant_content = [
            {"type": "text", "text": anime_data.description}
        ]
        messages = [
            {"role": "user", "content": user_content},
            {"role": "assistant", "content": assistant_content}
        ]

        return {"messages": messages}



    def __len__(self):
        return len(self.anime_data)


Определим экземпляр класса данных

In [8]:
anime_dataset = AnimeEpisodeCaptionDataset(
    dataset_path=dataset_path,
)
print(f'Successfully load {len(anime_dataset)} anime data')

Successfully load 614 anime data


Посмотрим пример выходных данных

In [9]:
example_message = anime_dataset[0]['messages']
example_message

[{'role': 'user',
  'content': [{'type': 'text',
    'text': 'The video genres are Shounen, Action, School, Super Power. '},
   {'type': 'text',
    'text': 'The main characters are All Might, Katsuki Bakugou, Tenya Iida, Izuku Midoriya, Ochako Uraraka. '},
   {'type': 'text', 'text': 'Caption the video. '},
   {'type': 'video',
    'path': '..\\src_dataset_creator\\dataset\\AniDataset_t614\\videos\\31964\\Boku no Hero Academia_S1_E1_720.mp4'}]},
 {'role': 'assistant',
  'content': [{'type': 'text',
    'text': 'The appearance of "quirks," newly discovered super powers, has been steadily increasing over the years, with 80 percent of humanity possessing various abilities from manipulation of elements to shapeshifting. This leaves the remainder of the world completely powerless, and Izuku Midoriya is one such individual.\n\nSince he was a child, the ambitious middle schooler has wanted nothing more than to be a hero. Izuku\'s unfair fate leaves him admiring heroes and taking notes on the

Посмотрим на входной текст после применения шаблона

In [11]:
example_instance = processor.apply_chat_template(
            example_message,
            add_generation_prompt=False,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        )
processor.batch_decode(example_instance["input_ids"])

You have used fast image processor with LANCZOS resample which not yet supported for torch.Tensor. BICUBIC resample will be used as an alternative. Please fall back to image processor if you want full consistency with the original model.


['<|im_start|>User: The video genres are Shounen, Action, School, Super Power. The main characters are All Might, Katsuki Bakugou, Tenya Iida, Izuku Midoriya, Ochako Uraraka. Caption the video. You are provided the following series of thirty-two frames from a 0:25:06 [H:MM:SS] video.\n\nFrame from 00:00:<fake_token_around_image><global-img><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><fake_token_around_image>\nFrame from 00:43:<fake_token_around_image><global-img><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><image><im

Разделим выборку на обучающую и тестовую

In [9]:
train_ds, eval_ds = random_split(
    anime_dataset,
    [0.95, 0.05],
    torch.Generator().manual_seed(42)
)
print(f"Train dataset length: {len(train_ds)}; eval dataset length: {len(eval_ds)}")

Train dataset length: 584; eval dataset length: 30


Создадим функцию по сборке набора данных во входной batch модели.

При решении задачи предсказания следующего токена формировании `lables` происходит на основе входных токенов, сдвинутых на 1 позицию вперёд. Во всех примерах реализации `collate_fn` отсутствует сдвиг вперёд. Причиной этого является уже встроенная функция сдвига в loss функцию (см. реализацию `transformers.loss.loss_utils.ForCausalLMLoss`).

При решении задачи инструктивного обучения по логике loss функция должна рассчитываться только от ответа ассистента. В большинстве обучающих примеров расчет происходит на всем выходе модели (за исключением специфичных токенов, таких как `<image>`) (прим. [официальная](https://github.com/huggingface/smollm/blob/main/vision/finetuning/SmolVLM2_Video_FT.ipynb) инструкция по fine-tuning SmolVLM2). Возможно, это происходит потому, что на текущий момент нет полноценно работающего инструмента внутри `transformers` для получения маски ответа ассистента. Текущая официальная реализация требует наличие маркера `{% generate %}` в шаблоне чата модели ([вопрос](https://github.com/huggingface/transformers/issues/33091) и [решение](https://github.com/huggingface/transformers/pull/30650)), что не подходит для всех моделей (Llama, Queen и сама SmolVLM от команды huggingface).

In [13]:
from torch.nn.utils.rnn import pad_sequence
from concurrent.futures import ThreadPoolExecutor


class ChatTemplateVLMCasualCollator:
    def __init__(
            self,
            processor: ProcessorMixin,
            thread_paralleling: bool = True,
            image_dtype=torch.float32
    ):
        self.processor = processor
        self._processor_assistant_mask_available = True
        self._thread_paralleling = thread_paralleling
        self._image_dtype = image_dtype

    def single_message_prepare(self, messages: list[dict[str, Any]]):
        # Преобразуем сообщение чата в набор признаков
        instance = processor.apply_chat_template(
            messages,
            add_generation_prompt=False,  # Отключаем добавление шаблона генерации продолжения
            tokenize=True,  # Токенизируем входной текст
            return_dict=True,  # Возврат всех данных, а не только "input_ids"
            return_assistant_tokens_mask=self._processor_assistant_mask_available,  # Возврат маски ответа ассистента
            # padding=True,  # Добавление padding для текста
            return_tensors="pt"
        )
        # Добавим токены выхода
        if "labels" not in instance:
            # Выход - входные токены модели (сдвиг на 1 токен внутри loss функции)
            labels = instance["input_ids"].clone()
            # Удалим специальные токены
            if hasattr(self.processor, "image_token_id"):
                labels[labels == self.processor.image_token_id] = -100
            # Проверим маску ассистента на некорретность
            if ("assistant_masks" in instance
                    and instance["assistant_masks"].element_size() > 0
                    and instance["assistant_masks"].sum() == 0
            ):
                warnings.warn(f"{processor.__class__.__name__} generate empty 'assistant_masks' output. Using assistant masked labels disabled")
                self._processor_assistant_mask_available = False
            # Применим маску ассистента к выходу
            if self._processor_assistant_mask_available:
                labels = labels.masked_fill(~instance["assistant_masks"].astype(bool), -100)
            instance["labels"] = labels

        return instance

    def __call__(self, examples: list[dict[str, list[dict[str, Any]]]]) -> dict[str, Any]:
        # Ввиду того, что apply_chat_template не работает с видео разной длины - обработаем каждое сообщение по отдельности
        if self._thread_paralleling:
            with ThreadPoolExecutor() as executor:
                instances = list(executor.map(
                    self.single_message_prepare,
                    [ex['messages'] for ex in examples])
                )
        else:
            instances = [
                self.single_message_prepare(ex['messages'])
                for ex in examples
            ]
        if len(instances) == 1:
            return {
                "input_ids": instances[0]["input_ids"],
                "attention_mask": instances[0]["attention_mask"],
                "labels": instances[0]["labels"],
                "pixel_values": instances[0]["pixel_values"].to(self._image_dtype)
            }

        # Объединим данные в единые тензоры
        out = {}
        for field_name, pad_value in (
                ("input_ids", processor.tokenizer.pad_token_id),
                ("attention_mask", 0),
                ("labels", -100)
        ):
            out[field_name] = pad_sequence(
                [inst[field_name].squeeze(0) for inst in instances],
                batch_first=True,
                padding_value=pad_value
            )

        # Объединим кадры
        # Получим требуемый общий размер объединенного тензора
        pvs = [inst["pixel_values"].squeeze(0) for inst in instances if "pixel_values" in inst]
        if pvs:  # there is at least one non-None pixel_values
            max_frames = max(pv.shape[0] for pv in pvs)
            max_h = max(pv.shape[-2] for pv in pvs)
            max_w = max(pv.shape[-1] for pv in pvs)
        else:
            max_h = max_w = processor.video_size['longest_edge']
            max_frames = 1

        padded_pixel_values = torch.zeros(
            (len(instances), max_frames, 3, max_h, max_w),
            dtype=self._image_dtype
        )
        for inst_idx, ex in enumerate(instances):
            pv = ex.get("pixel_values", None).squeeze(0)
            # Если есть изображения в инструкции
            if pv is not None:
                f, _, h, w = pv.shape
                padded_pixel_values[inst_idx, :f, :, :h, :w] = pv
        out["pixel_values"] = padded_pixel_values

        return out

In [14]:
collator = ChatTemplateVLMCasualCollator(processor=processor, thread_paralleling=True, image_dtype=model.dtype)

Проверим работоспособность сборщика

In [15]:
collate_data = collator([anime_dataset[i] for i in range(1)])
collate_data.keys()

return_assistant_tokens_mask==True but chat template does not contain `{% generation %}` keyword.
C:\Users\Domni\AppData\Local\Temp\ipykernel_29312\1345871012.py:40: UserWarning: SmolVLMProcessor generate empty 'assistant_masks' output. Using assistant masked labels disabled
  warnings.warn(f"{processor.__class__.__name__} generate empty 'assistant_masks' output. Using assistant masked labels disabled")


dict_keys(['input_ids', 'attention_mask', 'labels', 'pixel_values'])

## Обучим модель

Зададим параметры обучения

In [14]:
train_epochs = 1
batch_size = 1
target_batch_size = 32

model_name = model_id.split("/")[-1]
accumulation_steps = target_batch_size // batch_size

In [22]:
from trl import SFTTrainer, SFTConfig

In [23]:
sft_config = SFTConfig(
    num_train_epochs=train_epochs,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=accumulation_steps,
    warmup_steps=50,
    learning_rate=1e-4,
    weight_decay=0.01,
    logging_steps=50,
    save_strategy="steps",
    save_steps=250,
    save_total_limit=1,
    optim="adamw_torch_fused",
    bf16=True,
    output_dir=f"./{model_name}-anime-caption-sft",
    remove_unused_columns=False,
    report_to="tensorboard",
    dataloader_pin_memory=False,
    # Ускорение обучение за счёт компиляции модели
    # torch_compile=True,
    # torch_compile_backend="inductor",
    # torch_compile_mode="default",
    gradient_checkpointing=True,  # Leads to reduction in memory at slighly decrease in speed
    gradient_checkpointing_kwargs={"use_reentrant": False}, # Set gradient checkpointing to non-reentrant to avoid issues.
    dataset_kwargs={"skip_prepare_dataset": True}
)

In [24]:
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    data_collator=collator,
    train_dataset=train_ds,
    processing_class=processor,
)

In [25]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=19, training_loss=4.2534998743740433e+30, metrics={'train_runtime': 3903.7148, 'train_samples_per_second': 0.15, 'train_steps_per_second': 0.005, 'total_flos': 4479925849303296.0, 'train_loss': 4.2534998743740433e+30})

## Посмотрим на пример генерации

Напишем простенькую функцию для генерации ответа

In [23]:
def generate_caption(
        model,
        processor,
        conversation
):
    model.eval()
    message = conversation["messages"]
    # Уберём ответ агента, если он присутствует
    if message[-1]["role"] == "assistant":
        message = message[:-1]

    instance = processor.apply_chat_template(
        message,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt"
    ).to(model.device).to(model.dtype)

    with torch.no_grad():
        generated_ids = model.generate(**instance, do_sample=False, max_new_tokens=256)
    # print(instance["input_ids"].size())
    # print(generated_ids.size())
    # print(generated_ids[len(instance["input_ids"]):])

    generated_texts = processor.batch_decode(
        generated_ids[0][len(instance["input_ids"][0]):],
        skip_special_tokens=True,
    )
    generated_texts = "".join(generated_texts)

    return generated_texts

Загрузим исходную версию модели, чтобы сравнить качество генерации

In [11]:
vanila_model = AutoModelForImageTextToText.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    attn_implementation=attn_implementation,
    device_map=device,
)

Загрузим дообученную модель

In [15]:
trained_model = AutoModelForImageTextToText.from_pretrained(
    f"./{model_name}-anime-caption-sft/checkpoint-19",
    torch_dtype=torch_dtype,
    attn_implementation=attn_implementation,
    device_map=device,
)

Проведём сравнение

In [24]:
generate_test_message = eval_ds[1]
generate_test_anime_data = anime_dataset.get_anime_data_by_idx(eval_ds.indices[1])
print(
    f"\tTitle: {generate_test_anime_data.title}\n"
    f"\tTrue Caption:\n{generate_test_anime_data.description}\n\n"
    f"-------------------------------\n\n"
)

vanila_model_test_generation = generate_caption(vanila_model, processor, generate_test_message)
print(
    f"\tVanila model generation:\n{vanila_model_test_generation}\n\n"
    f"-------------------------------\n\n"
)

trained_model_test_generation = generate_caption(trained_model, processor, generate_test_message)
print(
    f"\tTrained model generation:\n{trained_model_test_generation}\n\n"
    f"-------------------------------\n\n"
)

	Title: Mushibugyo
	True Caption:
A menace of huge monster-like insects is plaguing the land of Edo. Too powerful to be subdued by ordinary folks, the creatures are hunted by the Insect Magistrates—a group of warriors who specialize in various secret arts and combat styles. To bolster their strength, they summon Genjuurou Tsukishima, the master swordsman from the Tsugaru Province.

Due to an unfortunate incident, however, Genjuurou is incapable of answering the call and sends his son, Jinbee, to serve in his stead. Determined to atone for the incident caused by his own cowardice, Jinbee agrees to travel to Edo and join the Insect Magistrates. Armed with his fiery spirit and unwavering resolve, Jinbee vows to become a stronger samurai and rid Edo of the insect threat once and for all.

-------------------------------


	Vanila model generation:
 The video features a series of scenes from an anime, each with distinct characters and settings. The first scene shows a character in a traditi

## Вывод

По результатам проделанной работы можно выделить следующее:
- Исходная модель SmolVLM2 хорошо справляется с описанием каждого кадра. Возможно, если провести prompt-engineering, то можно добиться хорошего результата по обобщению информации с кадров в одно цельное описание видео
- Обработка длинных видео даже достаточно малыми моделями является требовательной операцией по памяти графического процессора, т.к. обработка одного обучающего примера из 32 изображений моделью на 500 млн параметров потребовало 12 GB памяти CPU.
- Обучение на 1 эпохе занимает 1 час на RTX 4060 - основное ограничение идёт по памяти, из за чего CPU простаивает 99% времени
- После слишком короткого дообучения модель перестала справляться с генерацией предложений, что может быть вызвано слишком малым количеством пройденных эпох.
- Хоть провести полноценное обучение не хватило технических ресурсов, в рамках тетрадки был реализован полный pipeline дообучения на собственном наборе данных

# Extra: Нереализованные идеи

## Добавление поддержки assistant_mask в SmolVLM2

Добавим поддержку с получением маски ассистента ([вопрос](https://github.com/huggingface/transformers/issues/33091) и [решение](https://github.com/huggingface/transformers/pull/30650).

Для этого внедрим метки `{% generation %}` и `{% endgeneration %}` в шаблон сообщения ассистента.

Если в скором времени данную проблему решат более правильным способом - уберите этот блок

In [26]:
# target_template = "<|im_start|>{% for message in messages %}{{message['role'] | capitalize}}{% if message['content'][0]['type'] == 'image' %}{{':'}}{% else %}{{': '}}{% endif %}{% for line in message['content'] %}{% if line['type'] == 'text' %}{{line['text']}}{% elif line['type'] == 'image' %}{{ '<image>' }}{% endif %}{% endfor %}<end_of_utterance>\n{% endfor %}{% if add_generation_prompt %}{{ 'Assistant:' }}{% endif %}"
#
# if processor.chat_template == target_template:
#     processor.chat_template = (
#         "<|im_start|>"
#         "{% for message in messages %}"
#         "{{message['role'] | capitalize}}"
#         "{% if message['content'][0]['type'] == 'image' %}{{':'}}{% else %}{{': '}}{% endif %}"
#         "{% if message['role'] | lower != 'assistant' %}"
#         "{% for line in message['content'] %}"
#         "{% if line['type'] == 'text' %}{{line['text']}}"
#         "{% elif line['type'] == 'image' %}{{ '<image>' }}"
#         "{% endif %}"
#         "{% endfor %}"
#         "{% else %}"
#         "{% generation %}"  # Add generation marker
#         "{% for line in message['content'] %}{{line['text']}}{% endfor %}"
#         "{% endgeneration %}"  # Add endgeneration marker
#         "{% endif %}"
#         "<end_of_utterance>\n"
#         "{% endfor %}"
#         "{% if add_generation_prompt %}{{ 'Assistant:' }}{% endif %}"
#     )
# else:
#     warnings.warn('This resolve only applied for SmolVLMProcessor. For other model change template manually')

Проверим, что генерация вспомогательной маски работает корректно

In [27]:
# test_assistant_mask_message = [
#     {"role": "user", "content": [
#         {"type": "text", "text": "Caption the video."},
#         # {"type": "video", "path": "https://huggingface.co/datasets/hexuan21/VideoFeedback-videos-mp4/resolve/main/p/p110924.mp4"}
#     ]},
#     {"role": "assistant", "content": [
#         {"type": "text", "text": "A dog inside of a dog kennel on a patio."},
#     ]}
# ]
# print(processor.apply_chat_template(
#             test_assistant_mask_message,
#             add_generation_prompt=False,
#             tokenize=False,
#         ))
# test_assistant_mask_instance = processor.apply_chat_template(
#     test_assistant_mask_message,
#     add_generation_prompt=False,
#     tokenize=True,
#     return_dict=True,
#     return_tensors="pt",
#     return_assistant_tokens_mask=True,
# )
# assert test_assistant_mask_instance['assistant_masks'].sum() != 0
# del test_assistant_mask_message, test_assistant_mask_instance

Вывод: на текущей версии `transformers` даже после улучшений возвращается пустая маска